In [19]:
#imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import display
from rich.markdown import Markdown
from openai import OpenAI


In [5]:
#Connect to OpenAI using OpenAI API

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")
    

API key found and looks good so far!


In [6]:
#Create instance of OpenAI

openai = OpenAI()

In [7]:
#Scrap a website for summarizing

class Website:
    url:str
    title:str
    text:str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator="\n", strip=True)
                                     
                                     



In [8]:
site = Website("https://draft.dev/learn/technical-blog-platforms")
print(site.title)
print(site.text)

The Best Technical Blogging Platforms - Draft.dev
Why Us?
Lead Generation
Case Studies
Blog
Content Playbook
Company
Book Discovery Call
The Best Technical Blogging Platforms
June 29, 2024
•
karl
One of the first decisions any
developer marketing team
needs to make is
which technical blogging platform and content management system to use
. There are a lot of viable options, from well-known public platforms like Medium to specialized tools like Jekyll. You could even get your engineering team to build your own in-house platform on a headless CMS.
This question comes up a lot in
calls with clients
, so I wanted to round up some of the best writing platforms for running your technical blog, and share a little bit about why you might choose each of them. Note that we’ve also written detailed tutorials and comparisons for many of these, so be sure to check out the articles linked in each section.
Medium
One of the most popular blog platforms for developers out there is
Medium
. People often

In [9]:
#prompt

system_prompt = "You are an assistant that analyzes the content of a website \
and proivde a short summary, ignoring that might be navigation related \
Respond in markdown"

In [10]:
def user_prompt_for(website):
    user_prompt = f" You are looking at a website titled {website.title}"
    user_prompt += "\n The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [11]:
system_prompt

'You are an assistant that analyzes the content of a website and proivde a short summary, ignoring that might be navigation related Respond in markdown'

In [12]:
user_prompt_for(site)

' You are looking at a website titled The Best Technical Blogging Platforms - Draft.dev\n The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nWhy Us?\nLead Generation\nCase Studies\nBlog\nContent Playbook\nCompany\nBook Discovery Call\nThe Best Technical Blogging Platforms\nJune 29, 2024\n•\nkarl\nOne of the first decisions any\ndeveloper marketing team\nneeds to make is\nwhich technical blogging platform and content management system to use\n. There are a lot of viable options, from well-known public platforms like Medium to specialized tools like Jekyll. You could even get your engineering team to build your own in-house platform on a headless CMS.\nThis question comes up a lot in\ncalls with clients\n, so I wanted to round up some of the best writing platforms for running your technical blog, and share a little bit about why you might choose each of them. Note that 

In [21]:
print(user_prompt_for(site))


You are looking at a website titled The Best Technical Blogging Platforms - Draft.dev
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Why Us?
Lead Generation
Case Studies
Blog
Content Playbook
Company
Book Discovery Call
The Best Technical Blogging Platforms
June 29, 2024
•
karl
One of the first decisions any
developer marketing team
needs to make is
which technical blogging platform and content management system to use
. There are a lot of viable options, from well-known public platforms like Medium to specialized tools like Jekyll. You could even get your engineering team to build your own in-house platform on a headless CMS.
This question comes up a lot in
calls with clients
, so I wanted to round up some of the best writing platforms for running your technical blog, and share a little bit about why you might choose each of them. Note that we’ve also written detai

In [13]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
messages_for(site)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the content of a website and proivde a short summary, ignoring that might be navigation related Respond in markdown'},
 {'role': 'user',
  'content': ' You are looking at a website titled The Best Technical Blogging Platforms - Draft.dev\n The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nWhy Us?\nLead Generation\nCase Studies\nBlog\nContent Playbook\nCompany\nBook Discovery Call\nThe Best Technical Blogging Platforms\nJune 29, 2024\n•\nkarl\nOne of the first decisions any\ndeveloper marketing team\nneeds to make is\nwhich technical blogging platform and content management system to use\n. There are a lot of viable options, from well-known public platforms like Medium to specialized tools like Jekyll. You could even get your engineering team to build your own in-house platform on a headless CMS.\nTh

In [15]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [16]:
summarize("https://draft.dev/learn/technical-blog-platforms")

'# Summary of "The Best Technical Blogging Platforms - Draft.dev"\n\nDraft.dev provides an overview of various technical blogging platforms suitable for developer marketing teams. The article outlines prominent options such as:\n\n- **Medium**: Popular for its built-in audience and high domain authority, although it lacks control over ads and content management.\n  \n- **Gatsby**: A static site generator for developers using React.js, offering performance and SEO benefits, but requires technical skills for setup and content management.\n\n- **WordPress**: The most widely used platform, offering flexibility and a variety of plugins but can be complex for self-hosting.\n\n- **Jekyll**: A static site generator ideal for developers comfortable with Markdown on GitHub, but has a steeper learning curve.\n\n- **Hugo**: Another static site generator, known for speed and ease of setup, though it lacks plugin support.\n\n- **Ghost**: A minimalistic, Node.js-based platform for simple blogging, bu

In [20]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [23]:
display_summary("https://draft.dev/learn/technical-blog-platforms")


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                          Summary of The Best Technical Blogging Platforms - Draft.dev                           ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

The website provides an overview of various technical blogging platforms, offering recommendations tailored for    
developer marketing teams. It highlights the strengths and weaknesses of each platform to help users choose the    
right solution for their technical blog.                                                                           


                                                Featured Platforms                                                 

 1 Medium                                                                                                          
    • Popular for its built-in distribution and high domain authority.                                             
    • Recommended for syndication rather than exclusive use due to lack of control over ads and domain authority.  
 2 Gatsby                                                                                                          
    • An open-source static site generator focused on speed and SEO.                                               
    • Requires technical knowledge, as it’s based on React.js and has some limitations in content searching.       
 3 WordPress                                                                                                       
    • The most widely used blogging platform with a large plugin ecosystem.                                        
    • Differentiates between self-hosted (WordPress.org) and hosted (WordPress.com), with a preference for         
      self-hosting for greater control.                                                                            
 4 Jekyll                                                                                                          
    • A static site generator ideal for developers comfortable with HTML, CSS, and Markdown.                       
    • Offers free hosting via GitHub Pages, but requires technical skills to maintain.                             
 5 Hugo                                                                                                            
    • Another fast static site generator with an active community.                                                 
    • Easier to set up than Jekyll but lacks plugin support.                                                       
 6 Ghost                                                                                                           
    • A simple blogging platform focused on Markdown and custom themes.                                            
    • Requires web hosting or paid plans but offers live blog previewing features.                                 


                                                  Final Thoughts                                                   

The author emphasizes the evolving landscape of blogging platforms in the technical sphere and encourages feedback 
from users. The site also links to other lists of notable technical blogs across various categories.               

Overall, the content is designed to guide developers and marketing teams in selecting appropriate blogging         
platforms based on their needs and technical expertise.